# Estimating quartiles

# In brevi

The purpose of this notebook is to explore the ways that Python calculates quartiles. During the development of the anova_one_factor notebook, I discovered that Python, Libre Office, and Excel calculate quartiles in the same way, but that Minitab and an online source calculate them differently. I've discovered that there are at least nine ways to calculate quartiles.

# Data

Download the data file.

[estimating_quartiles](https://drive.google.com/open?id=1Nc_VFXo2SrsSdprfCmQYhLbJawAzKpH6)

# Methodology

Various data munging operations are performed using pandas.

In [1]:
from datetime import datetime
start_time = datetime.now()

In [2]:
# Import the required librairies.
import pandas as pd

/home/gilles/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gilles/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# Read the data file.
# y is the column of response values.
df = pd.read_csv('estimating_quartiles.csv')

In [4]:
def five_number_summary(data):
    global result
    min = data.min()
    q1 = data.quantile(.25, interpolation='midpoint')
    q2 = data.quantile(.50, interpolation='midpoint')
    q3 = data.quantile(.75, interpolation='midpoint')
    max = data.max()
    five_numbers = {
    'min' : [min],
    'q1' : [q1],
    'q2' : [q2],
    'q3' : [q3],
    'max' : [max]
    }
    result = pd.DataFrame(five_numbers)
    return result

In [5]:
# Five-number summary
a = {'y' : [five_number_summary]}
results = df.agg(a)
results

y                    
  five_number_summary                    
                  min   q1   q2    q3 max
0                   0  0.5  7.5  44.0  63

In [6]:
results.iloc[0,2]

7.5

In [7]:
end_time = datetime.now()
(end_time - start_time).total_seconds()

0.431991

# References

[Five-number summary](https://en.wikipedia.org/wiki/Five-number_summary)